# **Code**
#**참고한 공유 게시글**
        

*   유재성 KADE 님의 코드공유 [LightGBM + Optuna로 top 10안에 들어봅시다.]
    
    https://dacon.io/codeshare/2876

    optuna 사용법에 대해 알 수 있었습니다. 감사합니다.
<br/>

*   SDSTony 님의 코드공유 [정형데이터는 AutoML에 때려박고(?) 시작하자! PB Score: 0.77357]
    
    https://dacon.io/codeshare/1701

    PyCaret 사용법에 대해 알 수 있었습니다. 감사합니다.

# **각종 패키지 로드, 경로 설정**

In [ ]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# **Data preprocessing**




*   결측치에 0을 넣고 0을 평균으로 대체했습니다.
*   page에 머무는 시간에 해당하는 ('introelapse', 'testelapse', 'surveyelapse') 3가지 특성을 제거했습니다.
*   존재하는 이상치를 train data의 최빈값으로 대체했습니다.
*   원-핫 인코딩을 진행하기 위해 country 일부 값들을 train data의 최빈값으로 대체했습니다. 






In [ ]:
train=pd.read_csv('/gdrive/My Drive/Colab Notebooks/competition_data (1)/train.csv') 
test=pd.read_csv('/gdrive/My Drive/Colab Notebooks/competition_data (1)/test.csv')
submission=pd.read_csv('/gdrive/My Drive/Colab Notebooks/competition_data (1)/sample_submission.csv', index_col=0)

#결측치 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 70 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         15000 non-null  int64  
 1   Q1            14959 non-null  float64
 2   Q2            14931 non-null  float64
 3   Q3            14950 non-null  float64
 4   Q4            14929 non-null  float64
 5   Q5            14962 non-null  float64
 6   Q6            14952 non-null  float64
 7   Q7            14924 non-null  float64
 8   Q8            14952 non-null  float64
 9   Q9            14944 non-null  float64
 10  Q10           14928 non-null  float64
 11  Q11           14941 non-null  float64
 12  Q12           14933 non-null  float64
 13  Q13           14960 non-null  float64
 14  Q14           14964 non-null  float64
 15  Q15           14955 non-null  float64
 16  Q16           14967 non-null  float64
 17  Q17           14963 non-null  float64
 18  Q18           14937 non-nu

In [ ]:
#nan값 전부 0으로 삽입
train.fillna(0, inplace= True)
test.fillna(0, inplace= True)

In [ ]:
#상관관계가 적은 시간 특성, 인덱스 col을 제거

drop_list = ['introelapse', 'testelapse', 'surveyelapse', 'index',]

train.drop(drop_list, axis = 1, inplace = True)
test.drop(drop_list, axis = 1, inplace = True)

In [ ]:
train.columns

Index(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21',
       'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'country', 'TIPI1', 'TIPI2', 'TIPI3',
       'TIPI4', 'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1',
       'VCL2', 'VCL3', 'VCL4', 'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL9', 'VCL10',
       'VCL11', 'VCL12', 'VCL13', 'VCL14', 'VCL15', 'VCL16', 'education',
       'urban', 'gender', 'engnat', 'age', 'hand', 'religion', 'orientation',
       'voted', 'married', 'familysize', 'ASD', 'nerdiness'],
      dtype='object')

In [ ]:
#0값을 해당 특성의 train의 평균값으로 처리
processing_feature = ['gender', 'married', 'education', 'voted', 'urban', 'orientation', 'religion', 'engnat', 'ASD']

for pro in processing_feature:
    train.loc[train[pro] == 0, pro] = train[pro].mean()
    test.loc[test[pro] == 0, pro] = train[pro].mean()

In [ ]:
#age 이상치(80살 이상) 0으로 처리
train.loc[train['age'] > 80, 'age'] = 0
test.loc[test['age'] > 80, 'age'] = 0

In [ ]:
#age의 이상치는 train데이터의 평균값으로 처리
train.loc[train['age'] == 0, 'age'] = train['age'].mean()
test.loc[test['age'] == 0, 'age'] = train['age'].mean()

#familysize 6 이상인 값 최빈값으로 교체, 
train.loc[train['familysize'] > 6, 'familysize'] = train['familysize'].mode()[0]
test.loc[test['familysize'] > 6, 'familysize'] = train['familysize'].mode()[0]

In [ ]:
#훈련 데이터에 존재하는 상위 6개의 나라
best6_country = train['country'].value_counts().keys()[:6]

#train데이터 상위 6개 국가 제외 train데이터의 최빈값으로 처리
for i in range(len(train['country'])):
    if train['country'][i] not in best6_country or train['country'][i]==0:
        train['country'][i] = train['country'].mode()[0]

#test데이터 상위 6개 국가 제외 train데이터의 최빈값으로 처리
for i in range(len(test['country'])):
    if test['country'][i] not in best6_country or test['country'][i]==0:
        test['country'][i] = train['country'].mode()[0]

In [ ]:
#country 특성 원-핫인코딩 수행 (country 특성 세부 분할)
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [ ]:
train.columns

Index(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21',
       'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4',
       'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2',
       'VCL3', 'VCL4', 'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL9', 'VCL10',
       'VCL11', 'VCL12', 'VCL13', 'VCL14', 'VCL15', 'VCL16', 'education',
       'urban', 'gender', 'engnat', 'age', 'hand', 'religion', 'orientation',
       'voted', 'married', 'familysize', 'ASD', 'nerdiness', 'country_AUS',
       'country_CAN', 'country_DEU', 'country_GBR', 'country_PHL',
       'country_USA'],
      dtype='object')

In [ ]:
train.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,married,familysize,ASD,nerdiness,country_AUS,country_CAN,country_DEU,country_GBR,country_PHL,country_USA
0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,...,1.0,4.0,2.0,1,0,0,0,0,0,1
1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,...,2.0,4.0,2.0,1,0,0,0,0,0,1
2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,...,3.0,4.0,2.0,1,0,0,0,0,0,1
3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,...,1.0,2.0,2.0,1,0,0,0,0,0,1
4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,...,1.0,1.0,2.0,0,0,0,0,0,0,1


# **Hyperparameter Tuning: Optuna**
LGBM모델의 Hyperparameter Tuning을 진행하였습니다.


In [ ]:
!pip install --quiet optuna

     |████████████████████████████████| 308 kB 5.2 MB/s 
     |████████████████████████████████| 81 kB 8.3 MB/s 
     |████████████████████████████████| 209 kB 49.5 MB/s 
     |████████████████████████████████| 78 kB 7.8 MB/s 
     |████████████████████████████████| 112 kB 61.5 MB/s 
     |████████████████████████████████| 147 kB 62.0 MB/s 
     |████████████████████████████████| 49 kB 6.2 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import optuna
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from optuna.samplers import TPESampler
from optuna.pruners import SuccessiveHalvingPruner

In [ ]:
features = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21',
       'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4',
       'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2',
       'VCL3', 'VCL4', 'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL9', 'VCL10',
       'VCL11', 'VCL12', 'VCL13', 'VCL14', 'VCL15', 'VCL16', 'education',
       'urban', 'gender', 'engnat', 'age', 'hand', 'religion', 'orientation',
       'voted', 'married', 'familysize', 'ASD', 'country_AUS',
       'country_CAN', 'country_DEU', 'country_GBR', 'country_PHL',
       'country_USA']

target = ['nerdiness']

In [ ]:
df_trains = []
df_valids = []

skf = StratifiedKFold(n_splits=10, random_state=2022, shuffle=True)
for train_index, valid_index in skf.split(train[features], train[target]):
    df_train = train.loc[train_index]
    df_valid = train.loc[valid_index]
    df_trains.append(df_train)
    df_valids.append(df_valid)

    x_train = df_train[features]
    y_train = df_train[target]

    x_test = df_valid[features]
    y_test = df_valid[target]

In [ ]:
def accuracy(true, pred):
    return np.mean(true==pred)

def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 300, 824, step=1, log=True), 
        'max_depth': trial.suggest_int('max_depth', 10, 20, step=1, log=False), 
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1, log=True), 
        'n_estimators': trial.suggest_int('n_estimators', 1500, 3000, step=1, log=True), 
        "metric": "multi_auc",
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50, step=1, log=False), 
        'subsample': trial.suggest_uniform('subsample', 0.7, 1.0), 
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 1.0),
        'random_state': 2022
    }
    
    score = []
    for df_train, df_valid in zip(df_trains, df_valids):
        clf = LGBMClassifier(**params)
        clf.fit(x_train, y_train)
        
        pred = clf.predict_proba(df_valid[features])[:, 1]
        true = df_valid[target].values
        score.append(roc_auc_score(true, pred))
    score = np.mean(score)
    return score

# Hyperparameter Tuning
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=2022), pruner=SuccessiveHalvingPruner())
study.optimize(objective, n_trials=10)

[I 2022-08-26 11:18:54,950] A new study created in memory with name: no-name-9997bea5-c1a3-4b50-8cba-7b47c599f81b
[I 2022-08-26 11:21:50,508] Trial 0 finished with value: 0.8472915678710045 and parameters: {'num_leaves': 302, 'max_depth': 15, 'learning_rate': 0.00021885227715682474, 'n_estimators': 1552, 'class_weight': 'balanced', 'min_child_samples': 46, 'subsample': 0.8942356212229798, 'colsample_bytree': 0.969088936837299, 'reg_alpha': 0.7211349291646855, 'reg_lambda': 0.8313534207761898}. Best is trial 0 with value: 0.8472915678710045.
[I 2022-08-26 11:23:07,432] Trial 1 finished with value: 0.9871289336450279 and parameters: {'num_leaves': 692, 'max_depth': 19, 'learning_rate': 0.07432467297194512, 'n_estimators': 1936, 'class_weight': 'balanced', 'min_child_samples': 35, 'subsample': 0.9932588915263856, 'colsample_bytree': 0.7289299232913915, 'reg_alpha': 0.7442062122546155, 'reg_lambda': 0.2924994740953817}. Best is trial 1 with value: 0.9871289336450279.
[I 2022-08-26 11:28:35

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [ ]:
clfs = []
for df_train in df_trains:
    clf = LGBMClassifier(**study.best_params)
    clf.fit(df_train[features], df_train[target])
    pred_clf = clf.predict_proba(x_test)[:, 1]
    print(roc_auc_score(y_test, pred_clf))
    clfs.append(clf)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.8778259305880238


In [ ]:
clfs

[LGBMClassifier(colsample_bytree=0.7109969069989294,
                learning_rate=0.01047716753520929, max_depth=17,
                min_child_samples=11, n_estimators=2230, num_leaves=492,
                reg_alpha=0.2671838475497442, reg_lambda=0.2052238453808367,
                subsample=0.9346583262076185),
 LGBMClassifier(colsample_bytree=0.7109969069989294,
                learning_rate=0.01047716753520929, max_depth=17,
                min_child_samples=11, n_estimators=2230, num_leaves=492,
                reg_alpha=0.2671838475497442, reg_lambda=0.2052238453808367,
                subsample=0.9346583262076185),
 LGBMClassifier(colsample_bytree=0.7109969069989294,
                learning_rate=0.01047716753520929, max_depth=17,
                min_child_samples=11, n_estimators=2230, num_leaves=492,
                reg_alpha=0.2671838475497442, reg_lambda=0.2052238453808367,
                subsample=0.9346583262076185),
 LGBMClassifier(colsample_bytree=0.7109969069989294,
  

In [ ]:
pred = [clf.predict_proba(test[features]) for clf in clfs]

In [ ]:
real_pred = []

for i in range(10):
    for j in range(len(pred[i])):
        real_pred.append(pred[i][j][1])

In [ ]:
real_pred = []

for i in range(1):
    for j in range(len(pred[i])):
        real_pred.append(pred[i][j][1])

for i in range(2,10):
    for j in range(len(pred[i])):
        real_pred[j] += pred[i][j][1]
        if i==9:
            real_pred[j]= real_pred[j]/10

real_pred[:10]

[0.026400923758267764,
 0.8521683954927001,
 0.8934336643616095,
 0.681474496323301,
 0.8265500654265174,
 0.0035956373328560865,
 0.8781174815384766,
 0.8939905325290656,
 0.3177364368991601,
 0.2916122170030597]

In [ ]:
x_train = train.drop('nerdiness', axis = 1)
y_train = train['nerdiness']

# **2번째 model 선정**
Pycaret에서 제일 성능이 좋았던 model인 ExtraTreesClassifier을 사용했습니다.

In [ ]:
#extree model

extraTree_model = ExtraTreesClassifier(n_jobs=-1, random_state = 2022, n_estimators=3000)
extraTree_model.fit(x_train, y_train)
extraTree_pred = extraTree_model.predict_proba(test)[:, 1]

In [ ]:
real2_pred = list(extraTree_pred)

# **Ensemble**

LGBMClassifier (60%)
ExtraTreesClassifier (40%)

Ensemble을 진행하였습니다.

In [ ]:
result_pred = []

for i in range(len(real2_pred)):
    result_pred.append((real2_pred[i] * 0.6) + (real_pred[i] * 0.4))

result_pred

[0.010560369503307106,
 0.81526735819708,
 0.8679734657446438,
 0.6211897985293204,
 0.930620026170607,
 0.0014382549331424348,
 0.7316469926153907,
 0.8317962130116262,
 0.38969457475966407,
 0.38984488680122387,
 0.7152046053227361,
 0.8923958752052862,
 0.9540822789672336,
 0.3154608034788187,
 0.7979783055635066,
 0.2628202870369626,
 0.18127474117438253,
 0.6005328837273057,
 0.7573796043267966,
 0.7565688563107338,
 0.2461320682674445,
 0.23992666049988504,
 0.6553756876322647,
 0.48893069127100686,
 0.005879213677958662,
 0.715573177147929,
 0.7824116344266465,
 0.790107664703852,
 0.4819717013517111,
 0.0018193236723763396,
 0.009077251464804412,
 0.20856993912793226,
 0.9387618904469567,
 0.4192168938049741,
 0.7681752202817398,
 0.01345548903064605,
 0.42222038543274154,
 0.39593055650055636,
 0.6958164500961705,
 0.8238461623173325,
 0.7175418898715165,
 0.11111023079325356,
 0.7925559591533372,
 0.002215673944156806,
 0.8396985244134372,
 0.169552808201825,
 0.7611233279079

In [ ]:
submission['nerdiness'] = result_pred

In [ ]:
submission

,nerdiness
index,
0,0.010560
1,0.815267
2,0.867973
3,0.621190
4,0.930620
...,...
35447,0.947009
35448,0.734228
35449,0.911962


In [34]:
submission.to_csv('/gdrive/My Drive/LGBM(optuna, 10FOLD)_EXTRATR.csv')